In [ ]:
from newspaper import fulltext
from summarizer import Summarizer
from googlesearch import search 
from transformers import *
import requests
sentiment = pipeline('sentiment-analysis')
abst = TFAutoModelWithLMHead.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

In [ ]:
import re
article_url="https://analyticsindiamag.com/is-common-sense-common-in-nlp-models/"
article = fulltext(requests.get(article_url).text)
article=''.join([i for i in article.splitlines() if i])
article

In [ ]:
def get_summary(article_url,choice):
  article = fulltext(requests.get(article_url).text)
  article=''.join([i for i in article.splitlines() if i])
  if (choice):
    model = Summarizer()
    result = model(article, min_length=40)
    summary = ''.join(result)  
  else:#0 will return this
    inputs = tokenizer.encode("summarize: " + article, return_tensors="tf", max_length=512,truncation=True)
    summary = abst.generate(inputs, max_length=250, min_length=40, length_penalty=1.5, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary[0])
  return summary

In [ ]:
def get_url(query):
  urls = []
  for j in search(query, tld="com", num=10, stop=10, pause=2): 
      if j.count('wiki')==0:
          urls.append(j)
  return urls
urls=get_url("Farmer bill protest")
print(len(urls))

In [ ]:
ext_data={"Summary":[],"Source":[],"SS":[],"Label":[]}

for i in urls:
  summary=get_summary(i,1)
  s=sentiment(summary)
  print(f"summary\n{summary}\nby = {i}")
  ext_data['Summary'].append(summary)
  ext_data['Source'].append(i)
  ext_data['SS'].append(s[0]['label'])
  ext_data['Label'].append(s[0]['score'])
  print('*'*50)


In [ ]:
import pandas as pd
pd.DataFrame.from_dict(ext_data).to_csv("./Data/ext_summary.csv")

**<h2> Abstractive </h2>**

In [ ]:
abstractive_data={"Summary":[],"Source":[],"SS":[],"Label":[]}
for i in urls:
    summary = get_summary(i,0)
    s = sentiment(summary)
    print(f"summary\n{summary}\nby = {i}\nsentiment = {s}")
    abstractive_data['Summary'].append(summary)
    abstractive_data['Source'].append(i)
    abstractive_data['SS'].append(s[0]['label'])
    abstractive_data['Label'].append(s[0]['score'])
    print('*'*50)